![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps&file=Vertex+AI+Pipelines+-+Management.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FVertex%2520AI%2520Pipelines%2520-%2520Management.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Management.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Vertex%20AI%20Pipelines%20-%20Management.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

---
This is part a [series of notebook based workflows](./readme.md) that teach all the ways to use pipelines and experiments within Vertex AI. The suggested order and description/reason is:

||Notebook Workflow|Description|
|---|---|---|
||[Vertex AI Pipelines - Introduction](./Vertex%20AI%20Pipelines%20-%20Introduction.ipynb)|Introduction to pipelines with the console and Vertex AI SDK|
||[Vertex AI Pipelines - Components](./Vertex%20AI%20Pipelines%20-%20Components.ipynb)|An introduction to all the ways to create pipeline components from your code|
||[Vertex AI Pipelines - IO](./Vertex%20AI%20Pipelines%20-%20IO.ipynb)|An overview of all the type of inputs and outputs for pipeline components|
||[Vertex AI Pipelines - Control](./Vertex%20AI%20Pipelines%20-%20Control.ipynb)|An overview of controlling the flow of exectution for pipelines|
||[Vertex AI Pipelines - Secret Manager](./Vertex%20AI%20Pipelines%20-%20Secret%20Manager.ipynb)|How to pass sensitive information to pipelines and components|
||[Vertex AI Pipelines - Scheduling](./Vertex%20AI%20Pipelines%20-%20Scheduling.ipynb)|How to schedule pipeline execution|
|_**This Notebook**_|[Vertex AI Pipelines - Management](./Vertex%20AI%20Pipelines%20-%20Management.ipynb)|Managing, Reusing, and Storing pipelines and components|
||[Vertex AI Experiments](./Vertex%20AI%20Experiments.ipynb)|Understanding and using Vertex AI Experiments|

To discover these notebooks as part of an introduction to MLOps [start here](./readme.md)!

---

# Vertex AI Pipelines - Management

Pipeliens are made up of steps which are execution of components.  These components are made up of code, container, and instruction (inputs and outputs).  Managing the artifacts for components and pipelines extends the MLOps practice with with source control.  This notebook based workflow focuses on ways to manage both **components** and **pipelines**.  Managing also enables easier sharing, versioning, and use - with Vertex AI even non-code users can leverage pipelines directly in the console as pipeline template.


---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.artifactregistry_v1', 'google-cloud-artifact-registry'),
    ('kfp', 'kfp')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

## API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable artifactregistry.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
EXPERIMENT = 'pipeline-management'
SERIES = 'mlops'

# gcs bucket
GCS_BUCKET = PROJECT_ID

Packages

In [31]:
import os
import time
import importlib
from google.cloud import aiplatform
from google.cloud import artifactregistry_v1
import kfp
from typing import NamedTuple
from IPython.display import Markdown

Clients

In [9]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

# artifact registry client
ar_client = artifactregistry_v1.ArtifactRegistryClient()

parameters:

In [10]:
DIR = f"temp/{SERIES}-{EXPERIMENT}"

In [11]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [12]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## Setup Artifact Registry

[Artifact registry](https://cloud.google.com/artifact-registry/docs) organizes artifacts with repositories.  Each repository contains packages and is designated to hold a partifcular format of package: Docker images, Python Packages and [others](https://cloud.google.com/artifact-registry/docs/supported-formats#package).  There is even a registry type specifically for [Kubeflow pipeline templates](https://cloud.google.com/artifact-registry/docs/kfp?hl=en) which is the focus in this notebooks workflow. It specifically stores and managed `kfp` pipelines files (compiled pipeliens are YAML files).

### List Repositories

This may be empty if no repositories have been created for this project

In [13]:
for repo in ar_client.list_repositories(parent = f'projects/{PROJECT_ID}/locations/{REGION}'):
    print(repo.name)

projects/statmike-mlops-349915/locations/us-central1/repositories/gcf-artifacts
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-docker
projects/statmike-mlops-349915/locations/us-central1/repositories/statmike-mlops-349915-python


### Create/Retrieve KFP Repository

Create an Artifact Registry Repository to hold Docker Images created by this notebook.  First, check to see if it is already created by a previous run and retrieve it if it has.  Otherwise, create one named for this project.

In [19]:
try:
    kfp_repo = ar_client.get_repository(name = f'projects/{PROJECT_ID}/locations/{REGION}/repositories/{SERIES}')
except Exception:
    operation = ar_client.create_repository(
        parent = f'projects/{PROJECT_ID}/locations/{REGION}',
        repository_id = SERIES,
        repository = artifactregistry_v1.Repository(
            name = SERIES,
            format = artifactregistry_v1.Repository.Format.KFP
        )
    )
    kfp_repo = operation.result()

In [20]:
kfp_repo.name, kfp_repo.format_.name

('projects/statmike-mlops-349915/locations/us-central1/repositories/mlops',
 'KFP')

In [24]:
REPOSITORY = f"{REGION}-{kfp_repo.format_.name}.pkg.dev/{PROJECT_ID}/{kfp_repo.name.split('/')[-1]}"

In [25]:
REPOSITORY

'us-central1-KFP.pkg.dev/statmike-mlops-349915/mlops'

---
## Managing Components

Components are the building blocks of pipelines.  There are [multiple ways to construct components](./Vertex%20AI%20Pipelines%20-%20Components.ipynb) including using prebuilt components. For this example a simple lightweight Python component is constructed to show how the mechanics of managing a component. 

A key to keep in mind is that a component operates a remote function in a container.  The example uses a lightweight Python component and starts with a base image that is built at runtime with the provided Python function.  Another type of function is a container component where the code is already built into the container.  

### Create Pipeline Components

These are simple Python components, specifically lightweight Python components.  These are the same ones used in the [introduction workflow](./Vertex%20AI%20Pipelines%20-%20Introduction.ipynb).

Simple component with multiple inputs of different data types and outputs a dictionary.

In [26]:
@kfp.dsl.component(
    base_image = "python:3.11",
    packages_to_install = ["pandas"]
)
def example_parameters(
    in_str: str,
    in_int: int,
    in_float: float,
    in_bool: bool,
    in_list: list
) -> dict:
    results = dict(
        ex_str = in_str,
        ex_int = in_int,
        ex_float = in_float,
        ex_bool = in_bool,
        ex_list = in_list
    )
    return results

Simple component that take input parameters and outputs an artifact.

In [27]:
@kfp.dsl.component(
    base_image = "python:3.10",
    packages_to_install = ["pandas"]
)
def example_artifact(
    metadata: dict
) -> kfp.dsl.Artifact:
    ex_artifact = kfp.dsl.Artifact(
        metadata = metadata,
        uri = 'https://www.kubeflow.org/docs/components/pipelines/v2/data-types/artifacts/#artifact-types'
    )
    return ex_artifact

### Compile Components

Just as pipelines can be compiled into YAML, so can components.  The following does this compilation for both components created above.
- [`kfp.compiler`](https://kubeflow-pipelines.readthedocs.io/en/latest/source/compiler.html)

In [30]:
kfp.compiler.Compiler().compile(
    example_parameters,
    package_path = f'{DIR}/example_parameters.yaml'
)

kfp.compiler.Compiler().compile(
    example_artifact,
    package_path = f'{DIR}/example_artifact.yaml'
)

### Review Component YAML

The cell below print out the contents of one of the YAML files for a compiled component.  Note the following features:
- The `components` section has the details of the component including the input and output parameter names a data types
- The `deploymentSpec` has the container specified with:
    - `image` - the base image to use
    - `args` - the argument to run the specified function
    - `command` - in this case, a lightweight Python component, this has everything from the pip installs to the writing of the complete function to a file for execution

In [39]:
with open(f'{DIR}/example_parameters.yaml', 'r') as f:
    data = f.read()
Markdown(f"```yaml\n\n{data}\n```")

```yaml

# PIPELINE DEFINITION
# Name: example-parameters
# Inputs:
#    in_bool: bool
#    in_float: float
#    in_int: int
#    in_list: list
#    in_str: str
# Outputs:
#    Output: dict
components:
  comp-example-parameters:
    executorLabel: exec-example-parameters
    inputDefinitions:
      parameters:
        in_bool:
          parameterType: BOOLEAN
        in_float:
          parameterType: NUMBER_DOUBLE
        in_int:
          parameterType: NUMBER_INTEGER
        in_list:
          parameterType: LIST
        in_str:
          parameterType: STRING
    outputDefinitions:
      parameters:
        Output:
          parameterType: STRUCT
deploymentSpec:
  executors:
    exec-example-parameters:
      container:
        args:
        - --executor_input
        - '{{$}}'
        - --function_to_execute
        - example_parameters
        command:
        - sh
        - -c
        - "\nif ! [ -x \"$(command -v pip)\" ]; then\n    python3 -m ensurepip ||\
          \ python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1\
          \ python3 -m pip install --quiet --no-warn-script-location 'kfp==2.7.0'\
          \ '--no-deps' 'typing-extensions>=3.7.4,<5; python_version<\"3.9\"'  &&\
          \  python3 -m pip install --quiet --no-warn-script-location 'pandas' &&\
          \ \"$0\" \"$@\"\n"
        - sh
        - -ec
        - 'program_path=$(mktemp -d)


          printf "%s" "$0" > "$program_path/ephemeral_component.py"

          _KFP_RUNTIME=true python3 -m kfp.dsl.executor_main                         --component_module_path                         "$program_path/ephemeral_component.py"                         "$@"

          '
        - "\nimport kfp\nfrom kfp import dsl\nfrom kfp.dsl import *\nfrom typing import\
          \ *\n\ndef example_parameters(\n    in_str: str,\n    in_int: int,\n   \
          \ in_float: float,\n    in_bool: bool,\n    in_list: list\n) -> dict:\n\
          \    results = dict(\n        ex_str = in_str,\n        ex_int = in_int,\n\
          \        ex_float = in_float,\n        ex_bool = in_bool,\n        ex_list\
          \ = in_list\n    )\n    return results\n\n"
        image: python:3.11
pipelineInfo:
  name: example-parameters
root:
  dag:
    outputs:
      parameters:
        Output:
          valueFromParameter:
            outputParameterKey: Output
            producerSubtask: example-parameters
    tasks:
      example-parameters:
        cachingOptions:
          enableCache: true
        componentRef:
          name: comp-example-parameters
        inputs:
          parameters:
            in_bool:
              componentInputParameter: in_bool
            in_float:
              componentInputParameter: in_float
            in_int:
              componentInputParameter: in_int
            in_list:
              componentInputParameter: in_list
            in_str:
              componentInputParameter: in_str
        taskInfo:
          name: example-parameters
  inputDefinitions:
    parameters:
      in_bool:
        parameterType: BOOLEAN
      in_float:
        parameterType: NUMBER_DOUBLE
      in_int:
        parameterType: NUMBER_INTEGER
      in_list:
        parameterType: LIST
      in_str:
        parameterType: STRING
  outputDefinitions:
    parameters:
      Output:
        parameterType: STRUCT
schemaVersion: 2.1.0
sdkVersion: kfp-2.7.0

```

In [41]:
#with open(f'{DIR}/example_artifact.yaml', 'r') as f:
#    data = f.read()
#Markdown(f"```yaml\n\n{data}\n```")

### Loading A Compiled Component

Managing (saving, reusing, and sharing) a compiled component file is completed by being able to load the component directly for future pipelines.  This is accomplished using the [`kfp.components`](https://kubeflow-pipelines.readthedocs.io/en/latest/source/components.html) module which offers three functions for loading components compiled as YAML from either a file, a url or directly from a string:
- [`kfp.components.load_component_from_file`](https://kubeflow-pipelines.readthedocs.io/en/latest/source/components.html#kfp.components.load_component_from_file)
- [`kfp.components.load_component_from_url`](https://kubeflow-pipelines.readthedocs.io/en/latest/source/components.html#kfp.components.load_component_from_url)
- [`kfp.components.load_component_from_text`](https://kubeflow-pipelines.readthedocs.io/en/latest/source/components.html#kfp.components.load_component_from_text)

The following cell load the `example_parameters` component from file and creates a new local component named `imported_example_parameters`.

In [42]:
imported_example_parameters = kfp.components.load_component_from_file(f'{DIR}/example_parameters.yaml')

In [44]:
type(imported_example_parameters)

kfp.dsl.yaml_component.YamlComponent

---
## Managing Pipelines